# Clustering 
## Concepts

One often finds clusters in nature and datasets
- subsets which are similar to each other
- and dissimilar to others

Examples

![south England](london_lights_2012087.jpg)

Night lights in southern England

![double open cluster](DoubleCluster_h_chi_per.jpg)

Double open cluster h and $\chi$ Perseii

![age_vs_matches](football_cluster.png)

Ages vs. football matches visited.

- Humans are quite good at identifying clusters
- Identifying clusters in data science is an important tool.
    - Identifying clusters of consumers with similar attributes can be used to bombard them 
      with targeted advertising
    - Groups with certain health risks could be identified and advised to see a doctor.
- A number of clustering algorithms exist each one with pros and cons

First let's generate a sample with known cluster properties in $(x,y)$ space.


In [ ]:
import sklearn.datasets as skdat
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# define centres of three clusters
centres = [[-1., 0.], [1., -0.5], [0., 1.]]

# use make_blobs function to create dataset. 
# Points are normal distributed around the centres.
xy, nclust = skdat.make_blobs(1000, centers=centres, cluster_std=0.3)

What did we get?

In [ ]:
for coor, iclust in zip (xy, nclust):
    print(coor, iclust)

In [ ]:
x = xy[:,0]   # extract x and y vectors
y = xy[:,1]

# What did we get?
plt.figure()
plt.plot(x, y, "o", markersize=3)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# cluster by cluster
plt.figure(figsize=(5.0, 5.0))

plt.plot(x[nclust==0], y[nclust==0], "o", markersize=3)
plt.plot(x[nclust==1], y[nclust==1], "o", markersize=3)
plt.plot(x[nclust==2], y[nclust==2], "o", markersize=3)

plt.xlabel("x")
plt.ylabel("y")
plt.show()

## Clustering algorithms
### k-means

- k-means attempts to partition the samples into k clusters.
- k is given
- We'll get a set of cluster centres $\mathbf{S} = {S_1, S_2, S_3, \ldots S_k}$ each having coordinates $(S_{1,x}, S_{2,y})$
- One needs to measure the distance to the cluster centre. Usually the suqares of the Euklidian distance in vector space is used $(\mathbf{X} - \mathbf{S_i})^2$
- The minimisation problem:  $\min \sum_{i=1}^k \sum_{X \in S_i} (X-S_i)^2$ with $S_i$ begin the vector average over $X \in S_i$
- This is an iterative process

Implementation:
- visualise the data
- guess the number of clusters
- set-up the clusterer
- run the algorithm
- visualise and verify result
- possibly try another number of clusters


In [ ]:
# from sklearn import cluster
import sklearn.cluster as cluster

# set up the clusterer, 3 expected clusters
kmeans = cluster.KMeans(n_clusters=3)

# Fit the data, results are stored in the kmeans object
kmeans.fit(xy)     # fit done on x,y pairs

labels = kmeans.labels_
print(labels)    # labels is the number of the associated clusters of (x,y) points

# extract the estimated cluster centres
cen = kmeans.cluster_centers_
print(cen)

# plot using the labels to select colour
plt.figure(figsize=(5.0,5.0))

col = ["blue", "red", "green"]
for l in range(0,3):     # loop over the different labels
    plt.plot(x[labels==l], y[labels==l], "o", markersize=3, color=col[l])
    
# show cluster centres
for ic in range(3):
    xc, yc = cen[ic,:]
    plt.plot(xc, yc, "dk", markersize=10)
    
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Once we have fit (or trained) the clusterer cluster membership of new points can be tested

In [ ]:
print(kmeans.predict([[0.5, 0.5]]))

### Affinity propagation

- Affinity propagation uses the concept of *exemplars*
- It can determine the number of clusters on its own
- Consider a function $s$ quantifying the *similarity* between two points
- If $s(X_i, X_j) > s(X_i, X_k)$ then $X_i$ is more similar to $X_j$ then to $X_k$.
- Often used: the negative Euklidean distance is used to measure similarity $s(X_i, X_j) = -(X_i -X_j)^2$

The algorithm works by "passing" messenges between points.
- A response matrix **R** is set up with values of $R_{ik} = s(X_i, X_k)$
- An availability matrix matrix **A** describes how appropriate it would be for $X_i$ to pick $X_k$ as exmplar relative to all the other points

Iteration over two steps
1. First **R** is updated

$R_{ik} = s(X_i, X_k) - \max_{k\ne k'}(A_{ik} +s(X_i, X'_k))$

2. Followed by an update of **A**

$A_{ik} = min_{i\ne k}\left(0, R_{ik} + \sum_{i' \ne k}max(0, R_{i'k}\right)$

After a number of iterations we can identify *examplars*, i.e. representatives of a cluster using the criterion.

$R_{ik} + A_{ik} > 0$

This can be after a fixed number of iterations or when the process stops.

Defineing a criterion matrix **C** = **A** + **R** we can group together rows that share the same exemplar - defined as the column with the highest criterion value.

The implementation

In [ ]:
def make_colours():
    """ Creates a set of RGB representation of clusters. R, G,, B values are set in 
    steps of 0.25 and than all combinations of them are produced and returned as a list of
    tuples. """
    
    import itertools as iter
    
    r = (0.00, 0.35, 0.70)
    # g and b values are shifted to have more variation for the first 10-20 sets.
    g = (0.35, 0.70, 0.00)
    b = (0.70, 0.00, 0.35)
    
    # produce all combinations
    rgb = list(iter.product(r, g, b))
    
    return rgb

In [ ]:
rgb = make_colours()
print(rgb)

In [ ]:
xy, nclust = skdat.make_blobs(100, centers=centres, cluster_std=0.3)

x = xy[:,0]   # extract x and y vectors
y = xy[:,1]

# cluster by cluster
plt.figure(figsize=(5.0, 5.0))

plt.plot(x[nclust==0], y[nclust==0], "o", markersize=3)
plt.plot(x[nclust==1], y[nclust==1], "o", markersize=3)
plt.plot(x[nclust==2], y[nclust==2], "o", markersize=3)

plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# set up the clusterer
ap = cluster.AffinityPropagation(max_iter=2000, preference=-5)
# preference=-5  Preferences for each point - points with larger values of preferences are more 
# likely to be chosen as exemplars. Influences the number of clusters

# runnig it
ap.fit(xy)

labels = ap.labels_
cen = ap.cluster_centers_
# extract labels and centres
print("number of iterations", ap.n_iter_)
print("number of cluster centres", len(cen))

print(cen)
# plot using the labels to select colour
plt.figure(figsize=(5.0,5.0))
print(len(cen))
col = ["blue", "red", "green"]
for l in range(0, len(cen)):     # loop over the different labels
    plt.plot(x[labels==l], y[labels==l], "o", markersize=3, color=rgb[l])
    print(rgb[l])
print(cen)    
# show cluster centres
for ic in range(len(cen)):
    xc, yc = cen[ic,:]
    plt.plot(xc, yc, "dk", markersize=10)
    
plt.xlabel("x")
plt.ylabel("y")
plt.show()

### Agglomerative clustering

Type of *hierachical* clustering

- at first every point belongs to its own cluster - n clusters
- building up clusters by points being close together using some similarity measure $s$, often used $s(X_i, X_j) = -(X_i -X_j)^2$
- clustering clusters together
- clustering clusters of clusters together
- continue until complete hierachy has been build up.
- the number of clusters needs to be defined

![Hierachial clustering](hierachical_clustering.jpg)

Implementation

In [ ]:
xy, nclust = skdat.make_blobs(1000, centers=centres, cluster_std=0.3)

x = xy[:,0]   # extract x and y vectors
y = xy[:,1]

# cluster by cluster
plt.figure(figsize=(5.0, 5.0))

plt.plot(x[nclust==0], y[nclust==0], "o", markersize=3)
plt.plot(x[nclust==1], y[nclust==1], "o", markersize=3)
plt.plot(x[nclust==2], y[nclust==2], "o", markersize=3)

plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# set up the clusterer
ac = cluster.AgglomerativeClustering(n_clusters=3)

# carry out the fitting
ac.fit(xy)

labels = ac.labels_

# The clusterer does not return cluster centres, but they are easily computed
xcen = []
ycen = []
for ic in range(3):
    xc = np.average(x[labels==ic])
    yc = np.average(y[labels==ic])
    xcen.append(xc)
    ycen.append(yc)
print(xcen)

# plot using the labels to select colour
plt.figure(figsize=(5.0,5.0))

col = ["blue", "red", "green"]
for l in range(0,3):     # loop over the different labels
    plt.plot(x[labels==l], y[labels==l], "o", markersize=3, color=col[l])
    
# show cluster centres
for ic in range(3):
    plt.plot(xcen[ic], ycen[ic], "dk", markersize=10)
    
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# Extracting data from the web



In [ ]:
import pandas as pd

# URL for FTSE100 constituents
url = "https://www.londonstockexchange.com/indices/ftse-100/constituents/table"

page_ftse100 = pd.read_html(url)

What did we get?

In [ ]:
print(page_ftse100)

This is a list. Simple extraction of a dataframe

In [ ]:
df_ftse100 = page_ftse100[0]
print(df_ftse100.columns)
print(df_ftse100.describe())

In [ ]:
# second page
url = "https://www.londonstockexchange.com/indices/ftse-100/constituents/table?page=2"

page_ftse = pd.read_html(url)
df_ftse100b = page_ftse[0]

df_ftse100 = pd.concat([df_ftse100, df_ftse100b])
print(df_ftse100.describe())

What happens when more than one table appears?

In [ ]:
url = "https://en.wikipedia.org/wiki/Germany"
page_germ = pd.read_html(url)

In [ ]:
print(len(page_germ))
df = page_germ[2]
print(df)

Downloading and reading files from the web.

In [ ]:
url = "https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/heathrowdata.txt"

df_heathrow = pd.read_csv(url)

In [ ]:
import requests as req

r_heathrow = req.get(url)
data = r_heathrow.text
print(data[0:1000])

In [ ]:
import requests
import pandas as pd
import io

r_data = requests.get(url).content
df = pd.read_csv(io.StringIO(r_data.decode('utf-8')))

In [ ]:
print(df)